# Paris Real estate and venues analysis

## Importing Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import geocoder as geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [70]:
pd.options.display.float_format = "{:.2f}".format
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Importing data from DVF files 
https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/

In [48]:
filepath='/Users/yohanbizard/Documents/Documents/Documents/Cours/IBM Data science/Course 9 Capstone Project /RE_sales_paris_2019.csv'
df_paris=pd.read_csv(filepath)

In [49]:
df_paris.head()

,Date mutation,Valeur fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Price/sqm2
0,02/01/2019,180000.0,66,RUE,3510,DU FBOURG MONTMARTRE,75009,PARIS 09,2,Appartement,25,2,7200.00
1,02/01/2019,180000.0,9,PAS,8707,SAINT MICHEL,75017,PARIS 17,2,Appartement,25,2,7200.00
2,02/01/2019,99000.0,98,RUE,8847,SAUSSURE,75017,PARIS 17,2,Appartement,13,1,7615.38
3,02/01/2019,440000.0,36,AV,4754,D ITALIE,75013,PARIS 13,2,Appartement,57,2,7719.30
4,02/01/2019,635000.0,16,RUE,386,ANTOINE ROUCHER,75016,PARIS 16,2,Appartement,78,4,8141.03


In [50]:
df_paris.drop('Code voie',axis=1,inplace=True)

In [51]:
df_paris.head()

,Date mutation,Valeur fonciere,No voie,Type de voie,Voie,Code postal,Commune,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Price/sqm2
0,02/01/2019,180000.0,66,RUE,DU FBOURG MONTMARTRE,75009,PARIS 09,2,Appartement,25,2,7200.00
1,02/01/2019,180000.0,9,PAS,SAINT MICHEL,75017,PARIS 17,2,Appartement,25,2,7200.00
2,02/01/2019,99000.0,98,RUE,SAUSSURE,75017,PARIS 17,2,Appartement,13,1,7615.38
3,02/01/2019,440000.0,36,AV,D ITALIE,75013,PARIS 13,2,Appartement,57,2,7719.30
4,02/01/2019,635000.0,16,RUE,ANTOINE ROUCHER,75016,PARIS 16,2,Appartement,78,4,8141.03


In [52]:
#Obtain the list of the columns
df_paris.columns

Index(['Date mutation', 'Valeur fonciere', 'No voie', 'Type de voie', 'Voie',
       'Code postal', 'Commune', 'Code type local', 'Type local',
       'Surface reelle bati', 'Nombre pieces principales', 'Price/sqm2'],
      dtype='object')

In [211]:
#Rename columns in english
df_paris.rename(columns={'Date mutation':'Date', 
                         'Valeur fonciere':'price',
                         'No voie':'adress_num', 
                         'Type de voie':'street_type', 
                         'Voie':'street',
                         'Code postal':'postcode', 
                         'Commune':'borough', 
                         'Code type local':'estate_type_code',
                         'Type local':'estate_type',
                         'Surface reelle bati':'surface',
                         'Nombre pieces principales':'room_number',
                         'Price/sqm2':'price_per_sqm'},inplace=True)

In [212]:
df_paris.head()

,Date,price,adress_num,street_type,street,postcode,borough,estate_type_code,estate_type,surface,room_number,price_per_sqm
0,02/01/2019,180000.00,66,RUE,DU FBOURG MONTMARTRE,75009,PARIS 09,2,Appartement,25,2,7200.00
1,02/01/2019,180000.00,9,PAS,SAINT MICHEL,75017,PARIS 17,2,Appartement,25,2,7200.00
2,02/01/2019,99000.00,98,RUE,SAUSSURE,75017,PARIS 17,2,Appartement,13,1,7615.38
3,02/01/2019,440000.00,36,AV,D ITALIE,75013,PARIS 13,2,Appartement,57,2,7719.30
4,02/01/2019,635000.00,16,RUE,ANTOINE ROUCHER,75016,PARIS 16,2,Appartement,78,4,8141.03


In [55]:
df_paris['borough'].unique()

array(['PARIS 09', 'PARIS 17', 'PARIS 13', 'PARIS 16', 'PARIS 08',
       'PARIS 10', 'PARIS 14', 'PARIS 15', 'PARIS 18', 'PARIS 12',
       'PARIS 06', 'PARIS 11', 'PARIS 20', 'PARIS 19', 'PARIS 05',
       'PARIS 07', 'PARIS 01', 'PARIS 03', 'PARIS 04', 'PARIS 02',
       'ISSY-LES-MOULINEAUX', 'BOULOGNE-BILLANCOURT'], dtype=object)

In [56]:
#Drop rows for transaction that occured outside of Paris in ISSY-LES-MOULINEAUX and BOULOGNE-BILLANCOURT
df2=df_paris[df_paris['borough'] != 'BOULOGNE-BILLANCOURT']
df2=df2[df2['borough'] != 'ISSY-LES-MOULINEAUX']

In [57]:
df2['borough'].unique()

array(['PARIS 09', 'PARIS 17', 'PARIS 13', 'PARIS 16', 'PARIS 08',
       'PARIS 10', 'PARIS 14', 'PARIS 15', 'PARIS 18', 'PARIS 12',
       'PARIS 06', 'PARIS 11', 'PARIS 20', 'PARIS 19', 'PARIS 05',
       'PARIS 07', 'PARIS 01', 'PARIS 03', 'PARIS 04', 'PARIS 02'],
      dtype=object)

In [58]:
df2_paris=df2

In [59]:
df2_paris.dtypes

Date                 object
Valeur fonciere     float64
adress_num            int64
street_type          object
street               object
postcode              int64
borough              object
estate_type_code      int64
estate_type          object
surface               int64
room_number           int64
price_per_sqm       float64
dtype: object

### Getting a table with price per sqm2 per borough with postcode

In [71]:
df_borough = df2_paris[['postcode','borough','price_per_sqm']].groupby(['borough'],as_index=False).mean()
df_borough

,borough,postcode,price_per_sqm
0,PARIS 01,75001,51743.16
1,PARIS 02,75002,17455.89
2,PARIS 03,75003,46959.52
3,PARIS 04,75004,21889.82
4,PARIS 05,75005,16580.28
5,PARIS 06,75006,19264.15
6,PARIS 07,75007,20559.21
7,PARIS 08,75008,24198.68
8,PARIS 09,75009,20859.98
9,PARIS 10,75010,22931.74


In [72]:
def getLatLong(row):
    #print('post :{}'.format(row[:]))
    #print('neigh :{}'.format(row[1]))
    # initialize your variable to None
    lat_lng_coords = None
    search_query = '{}, Paris,FR'.format(row)
    # loop until you get the coordinates
    try:
        while(lat_lng_coords is None):
            #g = geocoder.here(search_query,app_id=app_id,app_code=app_code)
            g = geocoder.arcgis(search_query)
            lat_lng_coords = g.latlng
            #print('FIRST')
    except IndexError:
        latitude = 0.0
        longitude = 0.0
        print('BACKUP')
        return [latitude,longitude]

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print(latitude, longitude)
    return [latitude, longitude]

In [74]:
coords_list = df_borough['postcode'].apply(getLatLong).tolist()

48.86341500000003 2.3367708860000675
48.86771500000003 2.3430925840000327
48.86256000000003 2.3590470010000217
48.85427500000003 2.3614671030000522
48.84535000000005 2.35189172500003
48.84926500000006 2.33209944500004
48.85462000000007 2.3130611620000536
48.872934745000066 2.311248603000024
48.87699500000008 2.337893106000024
48.87615487100004 2.3623300000000427
48.85977500000007 2.378126286000054
48.835120000000074 2.444956778000062
48.83009000000004 2.3622827310000503
48.82979500000005 2.3238275120000367
48.841733558000044 2.2969748430000436
48.857120000000066 2.255970651000041
48.88707000000005 2.3062929050000207
48.89186500000005 2.3480935560000376
48.88394500000004 2.385625201000039
48.86239000000006 2.400828456000056


#### Add lat,long to the dataframe

In [76]:
df_borough[['Latitude','Longitude']]=pd.DataFrame(coords_list,columns=['Latitude', 'Longitude'])
df_borough.head()

,borough,postcode,price_per_sqm,Latitude,Longitude
0,PARIS 01,75001,51743.16,48.86,2.34
1,PARIS 02,75002,17455.89,48.87,2.34
2,PARIS 03,75003,46959.52,48.86,2.36
3,PARIS 04,75004,21889.82,48.85,2.36
4,PARIS 05,75005,16580.28,48.85,2.35


In [78]:
address = 'Paris'

geolocator = Nominatim(user_agent="ldn_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Paris are {}, {}.'.format(latitude, longitude))

The coordinates of Paris are 48.8566969, 2.3514616.


In [84]:
map_paris = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough in zip(df_borough['Latitude'], df_borough['Longitude'], df_borough['borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

#### Define Foursquare Credentials and Version

In [85]:
CLIENT_ID = 'WBLGYFNX3ZZTW1SQ32VFRGHER5IXK3ELLJYE25PSHCS4YRV3' # my Foursquare ID
CLIENT_SECRET = 'KQD0RLCH0SDKHENQEWFHFWPSIWLW4QBR0BYWCDWTNUSCMAUL' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WBLGYFNX3ZZTW1SQ32VFRGHER5IXK3ELLJYE25PSHCS4YRV3
CLIENT_SECRET:KQD0RLCH0SDKHENQEWFHFWPSIWLW4QBR0BYWCDWTNUSCMAUL


In [86]:
df_borough.loc[0, 'borough']

'PARIS 01'

In [88]:
borough_latitude = df_borough.loc[0, 'Latitude'] # neighborhood latitude value
borough_longitude = df_borough.loc[0, 'Longitude'] # neighborhood longitude value

borough_name = df_borough.loc[0, 'borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of PARIS 01 are 48.86341500000003, 2.3367708860000675.


#### let's explore top 100 venues around the 1st borough of Paris (PARIS 01) within a radius of 500 meters

In [89]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WBLGYFNX3ZZTW1SQ32VFRGHER5IXK3ELLJYE25PSHCS4YRV3&client_secret=KQD0RLCH0SDKHENQEWFHFWPSIWLW4QBR0BYWCDWTNUSCMAUL&v=20180605&ll=48.86341500000003,2.3367708860000675&radius=500&limit=100'

In [90]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f622e26aa469c1497b7f114'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Palais-Royal',
  'headerFullLocation': 'Palais-Royal, Paris',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 120,
  'suggestedBounds': {'ne': {'lat': 48.867915004500034,
    'lng': 2.343598520832485},
   'sw': {'lat': 48.85891499550003, 'lng': 2.32994325116765}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ba8b650f964a520f5e839e3',
       'name': 'Jardin du Palais Royal',
       'location': {'address': 'Palais Royal',
        'lat': 48.86494061245833,
        'lng': 2.3377275466918945,
        'labeledLatLngs': [{'label': 'display',
          'lat'

In [91]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [92]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/yohanbizard/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Jardin du Palais Royal,Garden,48.86,2.34
1,Palais Royal,Historic Site,48.86,2.34
2,Place du Palais Royal,Plaza,48.86,2.34
3,Comédie-Française,Theater,48.86,2.34
4,Place Colette,Plaza,48.86,2.34


In [96]:
nearby_venues.shape

(100, 4)

In [97]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['borough', 
                  'borough Latitude', 
                  'borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [98]:
paris_venues = getNearbyVenues(names=df_borough['borough'],
                                   latitudes=df_borough['Latitude'],
                                   longitudes=df_borough['Longitude']
                                  )


PARIS 01
PARIS 02
PARIS 03
PARIS 04
PARIS 05
PARIS 06
PARIS 07
PARIS 08
PARIS 09
PARIS 10
PARIS 11
PARIS 12
PARIS 13
PARIS 14
PARIS 15
PARIS 16
PARIS 17
PARIS 18
PARIS 19
PARIS 20


In [101]:
print(paris_venues.shape)
paris_venues

(1256, 7)


,borough,borough Latitude,borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,PARIS 01,48.86,2.34,Jardin du Palais Royal,48.86,2.34,Garden
1,PARIS 01,48.86,2.34,Palais Royal,48.86,2.34,Historic Site
2,PARIS 01,48.86,2.34,Place du Palais Royal,48.86,2.34,Plaza
3,PARIS 01,48.86,2.34,Comédie-Française,48.86,2.34,Theater
4,PARIS 01,48.86,2.34,Place Colette,48.86,2.34,Plaza
5,PARIS 01,48.86,2.34,La Clef Louvre Paris,48.86,2.34,Hotel
6,PARIS 01,48.86,2.34,Cibus,48.86,2.34,Italian Restaurant
7,PARIS 01,48.86,2.34,Thémaé,48.86,2.34,Spa
8,PARIS 01,48.86,2.34,Grand Hôtel du Palais Royal,48.86,2.34,Hotel
9,PARIS 01,48.86,2.34,Musée des Arts Décoratifs,48.86,2.33,Art Museum


#### Venues returned for each borough location

In [102]:
paris_venues.groupby('borough').count()

,borough Latitude,borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
borough,,,,,,
PARIS 01,100,100,100,100,100,100
PARIS 02,100,100,100,100,100,100
PARIS 03,100,100,100,100,100,100
PARIS 04,100,100,100,100,100,100
PARIS 05,100,100,100,100,100,100
PARIS 06,88,88,88,88,88,88
PARIS 07,59,59,59,59,59,59
PARIS 08,50,50,50,50,50,50
PARIS 09,100,100,100,100,100,100


In [103]:
print('There are {} uniques categories.'.format(len(paris_venues['Venue Category'].unique())))

There are 199 uniques categories.


#### Analyzing each borough Location

In [105]:
# one hot encoding
paris_onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough column back to dataframe
paris_onehot['borough'] = paris_venues['borough'] 

# move borough column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,borough,Afghan Restaurant,African Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Line,Bus Stop,Café,Cambodian Restaurant,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Provençal Restaurant,Pub,Ramen Restaurant,Recreation Center,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,PARIS 01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,PARIS 01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,PARIS 01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [106]:
paris_onehot.shape

(1256, 200)

#### Next, let's group rows by borough and by taking the mean of the frequency of occurrence of each category

In [109]:
paris_grouped = paris_onehot.groupby('borough').mean().reset_index()
paris_grouped

,borough,Afghan Restaurant,African Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,Bagel Shop,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Line,Bus Stop,Café,Cambodian Restaurant,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,New American Restaurant,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Provençal Restaurant,Pub,Ramen Restaurant,Recreation Center,Resort,Restaurant,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Snack Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,PARIS 01,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.01,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.03,0.00,0.01,0.02,0.01,0.00,0.00,0.00,0.01,0.05,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.09,0.00,0.01,0.02,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.09,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.06,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.02,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.01,0.00,0.00
1,PARIS 02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01

#### Let's confirm the new size

In [110]:
paris_grouped.shape

(20, 200)

#### Let's print each borough along with the top 5 most common venues

In [111]:
num_top_venues = 5

for hood in paris_grouped['borough']:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----PARIS 01----
                 venue  freq
0    French Restaurant  0.09
1  Japanese Restaurant  0.09
2                Hotel  0.06
3                Plaza  0.06
4          Coffee Shop  0.05


----PARIS 02----
                venue  freq
0   French Restaurant  0.11
1        Cocktail Bar  0.06
2            Wine Bar  0.05
3  Italian Restaurant  0.05
4              Bistro  0.04


----PARIS 03----
          venue  freq
0   Coffee Shop  0.05
1  Burger Joint  0.04
2        Bistro  0.04
3   Art Gallery  0.04
4        Bakery  0.04


----PARIS 04----
                venue  freq
0   French Restaurant  0.14
1               Hotel  0.05
2      Clothing Store  0.05
3      Ice Cream Shop  0.04
4  Italian Restaurant  0.04


----PARIS 05----
                venue  freq
0   French Restaurant  0.17
1                 Bar  0.06
2               Hotel  0.05
3  Italian Restaurant  0.04
4                Café  0.03


----PARIS 06----
                venue  freq
0   French Restaurant  0.11
1  Italian Restaurant 

#### Let's put that into a *pandas* dataframe

In [112]:
# sorting venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
#Create a dataframe and display the top 10 venues for each borough.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['borough'] = paris_grouped['borough']

for ind in np.arange(paris_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted.head()

,borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS 01,French Restaurant,Japanese Restaurant,Hotel,Plaza,Coffee Shop,Italian Restaurant,Café,Historic Site,Bar,Art Museum
1,PARIS 02,French Restaurant,Cocktail Bar,Wine Bar,Italian Restaurant,Bistro,Salad Place,Hotel,Thai Restaurant,Bakery,Furniture / Home Store
2,PARIS 03,Coffee Shop,Bakery,Bistro,Burger Joint,French Restaurant,Art Gallery,Clothing Store,Café,Cocktail Bar,Sandwich Place
3,PARIS 04,French Restaurant,Hotel,Clothing Store,Italian Restaurant,Ice Cream Shop,Bakery,Plaza,Wine Bar,Tapas Restaurant,Coffee Shop
4,PARIS 05,French Restaurant,Bar,Hotel,Italian Restaurant,Coffee Shop,Bakery,Creperie,Café,Wine Bar,Pub


#### Cluster Neighborhoods

Run *k*-means to cluster the borough into 5 clusters.

In [116]:
# set number of clusters
kclusters = 5

paris_grouped_clustering = paris_grouped.drop('borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 4, 4, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each borough.

In [117]:
# add clustering labels
borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

paris_merged = df_borough

# merge paris_grouped with the paris venues dataframe to add latitude/longitude for each borough
paris_merged = paris_merged.join(borough_venues_sorted.set_index('borough'), on='borough')

paris_merged.head() # check the last columns!

,borough,postcode,price_per_sqm,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS 01,75001,51743.16,48.86,2.34,1,French Restaurant,Japanese Restaurant,Hotel,Plaza,Coffee Shop,Italian Restaurant,Café,Historic Site,Bar,Art Museum
1,PARIS 02,75002,17455.89,48.87,2.34,1,French Restaurant,Cocktail Bar,Wine Bar,Italian Restaurant,Bistro,Salad Place,Hotel,Thai Restaurant,Bakery,Furniture / Home Store
2,PARIS 03,75003,46959.52,48.86,2.36,1,Coffee Shop,Bakery,Bistro,Burger Joint,French Restaurant,Art Gallery,Clothing Store,Café,Cocktail Bar,Sandwich Place
3,PARIS 04,75004,21889.82,48.85,2.36,1,French Restaurant,Hotel,Clothing Store,Italian Restaurant,Ice Cream Shop,Bakery,Plaza,Wine Bar,Tapas Restaurant,Coffee Shop
4,PARIS 05,75005,16580.28,48.85,2.35,1,French Restaurant,Bar,Hotel,Italian Restaurant,Coffee Shop,Bakery,Creperie,Café,Wine Bar,Pub


In [118]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['borough'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [203]:
#Define a function to display the average price per square meter
def price(bn):
    x=round(paris_merged['price_per_sqm'][bn-1],2)
    print('The average price per sqm in PARIS',bn,'is :',x,'€')

Cluster 1 Sport/Leisure oriented borough

In [119]:
paris_merged.loc[paris_merged['Cluster Labels'] == 0, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,75012,0,Theater,Stadium,Performing Arts Venue,Playground,Fast Food Restaurant,Bike Trail,Recreation Center,Botanical Garden,Restaurant,Comedy Club


In [204]:
price(12)

The average price per sqm in PARIS 12 is : 11780.43 €


Cluster 2 : Tourism,night life and entertainment

In [120]:
paris_merged.loc[paris_merged['Cluster Labels'] == 1, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75001,1,French Restaurant,Japanese Restaurant,Hotel,Plaza,Coffee Shop,Italian Restaurant,Café,Historic Site,Bar,Art Museum
1,75002,1,French Restaurant,Cocktail Bar,Wine Bar,Italian Restaurant,Bistro,Salad Place,Hotel,Thai Restaurant,Bakery,Furniture / Home Store
2,75003,1,Coffee Shop,Bakery,Bistro,Burger Joint,French Restaurant,Art Gallery,Clothing Store,Café,Cocktail Bar,Sandwich Place
3,75004,1,French Restaurant,Hotel,Clothing Store,Italian Restaurant,Ice Cream Shop,Bakery,Plaza,Wine Bar,Tapas Restaurant,Coffee Shop
4,75005,1,French Restaurant,Bar,Hotel,Italian Restaurant,Coffee Shop,Bakery,Creperie,Café,Wine Bar,Pub
5,75006,1,French Restaurant,Chocolate Shop,Hotel,Italian Restaurant,Restaurant,Plaza,Fountain,Pub,Tea Room,Athletics & Sports
8,75009,1,French Restaurant,Hotel,Bakery,Japanese Restaurant,Bistro,Wine Bar,Lounge,Cocktail Bar,Restaurant,Gym / Fitness Center
9,75010,1,French Restaurant,Café,Bistro,Coffee Shop,Asian Restaurant,Bakery,Pizza Place,Wine Bar,Hotel,Italian Restaurant
10,75011,1,French Restaurant,Café,Restaurant,Wine Bar,Vegetarian / Vegan Restaurant,Pastry Shop,Italian Restaurant,Bistro,Cocktail Bar,Afghan Restaurant
12,75013,1,Vietnamese Restaurant,Chinese Restaurant,Hotel,Asian Restaurant,French Restaurant,Thai Restaurant,Convenience Store,Sushi Restaurant,Creperie,Print Shop


In [206]:
price(1)
price(2)
price(3)
price(4)
price(5)
price(6)
price(9)
price(10)
price(11)
price(13)
price(15)
price(18)
price(20)

The average price per sqm in PARIS 1 is : 51743.16 €
The average price per sqm in PARIS 2 is : 17455.89 €
The average price per sqm in PARIS 3 is : 46959.52 €
The average price per sqm in PARIS 4 is : 21889.82 €
The average price per sqm in PARIS 5 is : 16580.28 €
The average price per sqm in PARIS 6 is : 19264.15 €
The average price per sqm in PARIS 9 is : 20859.98 €
The average price per sqm in PARIS 10 is : 22931.74 €
The average price per sqm in PARIS 11 is : 15371.6 €
The average price per sqm in PARIS 13 is : 14728.84 €
The average price per sqm in PARIS 15 is : 17322.84 €
The average price per sqm in PARIS 18 is : 25410.04 €
The average price per sqm in PARIS 20 is : 27052.85 €


Cluster 3 : Family and leisure borough

In [121]:
paris_merged.loc[paris_merged['Cluster Labels'] == 2, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,75016,2,Pool,Lake,Bike Rental / Bike Share,Circus,Yoga Studio,Falafel Restaurant,Frozen Yogurt Shop,French Restaurant,Fountain,Food & Drink Shop


In [207]:
price(16)

The average price per sqm in PARIS 16 is : 18104.15 €


Cluster 4 : International, tourism and restaurants

In [122]:
paris_merged.loc[paris_merged['Cluster Labels'] == 3, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,75014,3,French Restaurant,Hotel,Italian Restaurant,Thai Restaurant,Bistro,Food & Drink Shop,Café,Fast Food Restaurant,Tea Room,Bakery


In [208]:
price(14)

The average price per sqm in PARIS 14 is : 16035.64 €


Cluster 5 : restaurant, supermarkets and tourism

In [123]:
paris_merged.loc[paris_merged['Cluster Labels'] == 4, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,75007,4,Hotel,French Restaurant,Café,Plaza,History Museum,Italian Restaurant,Garden,Coffee Shop,Historic Site,Shop & Service
7,75008,4,French Restaurant,Hotel,Art Gallery,Spa,Japanese Restaurant,Cocktail Bar,Sporting Goods Shop,Department Store,Cosmetics Shop,Corsican Restaurant
16,75017,4,French Restaurant,Hotel,Italian Restaurant,Japanese Restaurant,Bakery,Café,Plaza,Bistro,Pastry Shop,Restaurant
18,75019,4,French Restaurant,Supermarket,Bakery,Hotel,Beer Garden,Concert Hall,Bike Rental / Bike Share,Scenic Lookout,Historic Site,Bar


In [209]:
price(7)
price(8)
price(17)
price(19)

The average price per sqm in PARIS 7 is : 20559.21 €
The average price per sqm in PARIS 8 is : 24198.68 €
The average price per sqm in PARIS 17 is : 15631.0 €
The average price per sqm in PARIS 19 is : 36809.44 €
